In [19]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
dpath='D:/kaggle-10-most-popular-dataset/3-Digit Recognizer//'

In [115]:
train=pd.read_csv(dpath+'train.csv')
test=pd.read_csv(dpath+'test.csv')

In [154]:
train.head(5)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [155]:
test.head(5)

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [156]:
print(np.shape(train))
print(np.shape(test))

(42000, 785)
(28000, 784)


In [157]:
Ytrain=np.array(train.copy()['label'])

In [158]:
np.shape(Ytrain)
Ytrain.reshape(42000,-1)

array([[1],
       [0],
       [1],
       ...,
       [7],
       [6],
       [9]], dtype=int64)

In [159]:
Xtrain=np.array(train.drop(['label'],axis=1))
np.shape(Xtrain)
np.shape(Ytrain)

(42000,)

In [160]:
import tensorflow as tf

In [174]:
#2.定义超参数和placeholder
#超参数
learning_rate=0.05
epochs=10
batch_size=100

#placeholder
#输入图片为28 x 28像素=784
x=tf.placeholder(tf.float32,[None,784])  #None代表任意值，特别对应tensor数目
#输出为0-9的one-hot编码
y=tf.placeholder(tf.float32,[None,10])

In [175]:
#3.定义参数w和b
# hidden layer 
w1=tf.Variable(tf.random_normal([784,300],stddev=0.03),name='w1')
b1=tf.random_normal([300],name='b1')
#output layer
w2=tf.Variable(tf.random_normal([300,10],stddev=0.03),name='w2')
b2=tf.random_normal([10],name='b2')

In [176]:
#4.构造隐层网络
#hidden layer
hidden_out=tf.add(tf.matmul(x,w1),b1)   #相当于矩阵乘法 z=w1x+b1
hidden_out=tf.nn.relu(hidden_out)       #h=relu(z)

In [177]:
#5.构造输出（预测值）
y_=tf.nn.softmax(tf.add(tf.matmul(hidden_out,w2),b2))      #相当于矩阵乘法 u=w2h+b2
#对于单标签多酚类任务，输出层的激活函数都是tf.nn.relu()        #y_=softmax(u)

In [178]:
#定义loss 损失函数
#多分类label，损失设定为交叉熵
#分为两步：
#1.对n个标签计算交叉熵
#2.对m个样本取平均
y_clipped=tf.clip_by_value(y_,1e-10,0.999999)                                                       #压缩预测值y_在0-1之间 
cross_entropy=-tf.reduce_mean(tf.reduce_sum(y*tf.log(y_clipped)+(1-y)*tf.log(1-y_clipped),axis=1))  #计算交叉熵（损失函数）

In [179]:
#7.定义优化算法
#创建优化器，确定优化目标
optimizer=tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cross_entropy)  #优化算法为梯度下降

In [180]:
#8.定义初始化operation和准确率node
#init operator
init_op=tf.global_variables_initializer()

#创建准确率节点
correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))            #correct_prediction会返回一个m x 1的tensor，tensor的值为True/False表示是否预测正确
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

#tf.argmax(vector, 1)：返回的是vector中的最大值的索引号，如果vector是一个向量，那就返回一个值，如果是一个矩阵，那就返回一个向量，这个向量的每一个维度都是相对应矩阵行的最大值元素的索引号。

In [181]:
np.shape(Xtrain)


(42000, 784)

In [182]:
Ytrain

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [183]:
Ytrain=Ytrain.reshape(42000,-1)
np.shape(Ytrain)
def convert_to_one_hot(y, C):
    return np.eye(C)[y.reshape(-1)].T
Ytrain=convert_to_one_hot(Ytrain,10)

IndexError: arrays used as indices must be of integer (or boolean) type

In [171]:
Ytrain

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [184]:
def next_batch(train_data, train_target, batch_size):  
    index = [ i for i in range(0,len(train_target)) ]  
    np.random.shuffle(index);  
    batch_data = []; 
    batch_target = [];  
    for i in range(0,batch_size):  
        batch_data.append(train_data[index[i]]);  
        batch_target.append(train_target[index[i]])  
    return batch_data, batch_target

In [186]:
#9.开始训练
# 创建session
with tf.Session() as sess:
    # 变量初始化
    sess.run(init_op)
    total_batch = int(len(Ytrain) / batch_size)
    for epoch in range(epochs):
        avg_cost = 0
        for i in range(total_batch):
            batch_x, batch_y =next_batch(Xtrain,Ytrain,batch_size=batch_size)
            _,c=sess.run([optimizer, cross_entropy], feed_dict={x: batch_x, y: batch_y})
            avg_cost += c / total_batch
        print("Epoch:", (epoch + 1), "cost = ", "{:.3f}".format(avg_cost))
    print(sess.run(accuracy, feed_dict={x: Xtrain, y: Ytrain}))

Epoch: 1 cost =  33.171
Epoch: 2 cost =  33.105
Epoch: 3 cost =  33.327
Epoch: 4 cost =  33.265
Epoch: 5 cost =  33.184
Epoch: 6 cost =  33.104
Epoch: 7 cost =  33.214
Epoch: 8 cost =  33.161
Epoch: 9 cost =  33.129
Epoch: 10 cost =  33.381
0.06502381
